In [ ]:
# First, we write the function for creating box
# This is the code to make a box around a vector v = [v1 v2 ... vn]^T with parameters r = [r1 r2 ... rn]^T
def make_box(v, r):
    n = len(v)
    zeros = [0]*(n+1)
    mat = []
    # Adding the constraints for positive values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)
    # Adding the constraints for negative values
    for r1 in r:
        zeros[0] = r1
        mat.append(zeros*1)   
    # print('mat zeros = ', mat) # [[0.1,0,0], [0.1,0,0], [0.1,0,0], [0.1,0,0]]
    
    for i in range(n):
        mat[i][i+1] = 1 # [[0.1,1,0], [0.1,0,1], [0.1,0,0], [0.1,0,0]]
    # print('mat pos = ', mat)
    j = 1
    for i in range(n,2*n):
        mat[i][j] = -1 # # [[0.1,1,0], [0.1,0,1], [0.1,-1,0], [0.1,0,-1]]
        j += 1
    # print('mat for constraints = ', mat)
    Box = Polyhedron(ieqs=mat, backend='ppl', base_ring=QQ)
    Box_vertices = np.array(Box.vertices_list())
    return Box_vertices

# Minkowski sum function
def minkowski_pol(V1, V2):
    V3 = []
    for v1 in V1:
        for v2 in V2:
            n = v1.size
            v = np.zeros(n)
            for i in range(n):
                v[i] = v1[i] + v2[i]
            V3.append(v)
    # print(V3)
    V4 = np.array(V3)
    return V4

def convex_hull(V1, V2):
    V2 = np.append(V1,V2)
    return V2

def linear_trans(M, V):
    # V1 = P1.vertices_list()
    # V2 = P2.vertices_list()
    V1 = []
    for v in V:
        x =  np.matmul(M,v)
        x = np.array(x)
        V1.append(x)
        # print(V1)
    V2 = np.array(V1)
    return V2


In [ ]:
import numpy as np

# We first import the time module to compute runtime
import time
start_time = time.time()


# Select the file to take input for algorithm 1
v = input("Enter the example number to run algorithm 2: ")
exp_num = v
# Open the input file
if (int(v)==1):
    input_file = open(r"inputs_new/ex1_dummy.txt")
elif (int(v)==2):
    input_file = open(r"inputs_new/ex2_balance_system.txt")
elif (int(v)==3):
    input_file = open(r"inputs_new/ex3_3deg_quadcopter.txt")
elif (int(v)==4):
    input_file = open(r"inputs_new/ex4_6deg_quadcopter.txt")

# Execute the input file to get A, B, K, X0, U0
exec(input_file.read())
input_file.close()

# Create the box
v = [0]*(2*n)
r = [ep]*n
zeros = [0]*n
rbox = zeros+r
Box = make_box(v,rbox)
# print('Box = ', Box.vertices())

# Converting K into K = [[I 0] [0 K]]
xn, un = n, u
K1 = [[0 for i in range(xn)]for j in range(xn)]
KI = np.identity(xn)
K1 = np.hstack((KI, K1))
K2 = [[0 for i in range(xn)] for j in range(un)]
K2 = np.hstack((K2,K))
K = Matrix(np.vstack((K1,K2)))

# At first, we define C = [A B]
C = Matrix(np.hstack((A,B)))

# Next, we have input P0 = [X0 U0]^T
P0 = np.array([np.hstack((x,u)) for x in X0 for u in U0])

# We define the matrix M as follows,
I = np.identity(xn)
M = np.array(np.vstack((I,I)))

In [14]:
start_time = time.time()

# Opening a file to write the outputs
exp_string = "outputs/alg2_new/ex" + exp_num + ".csv"
f = open(exp_string, "a")
f.write('\nNew Experiment:\n')


# Running algorithm 2
Pc = P0
iter = 50
ep = [10]
for e in ep:
    try:
        msg = '\n ep = '+str(e)+'\n'
        print(msg)
        f.write(msg)
        msg = 'Iteration, Num_Ver(Xc), Volume(Xc), Runtime\n'
        print(msg)
        f.write(msg)
        for i in range(iter):        
            # print('i = ', i, file=f)
            Xc = linear_trans(C,Pc)
            # print('Number_of_Cons(Xc) = ', len(list(Xc.Hrepresentation())), file=f)
            Xc_twice = linear_trans(M, Xc)
            # print('Number_of_Cons(Xc_twice) = ', len(list(Xc_twice.Hrepresentation())), file=f)
            if e:
                v = [0]*(2*n)
                r = [e]*n
                zeros = [0]*n
                rbox = zeros+r
                Box = make_box(v,rbox)
                Xh = minkowski_pol(Xc_twice,Box)
            else:
                Xh = Xc_twice
            # print('Number_of_Cons(Xh) = ', len(list(Xh.Hrepresentation())), file=f)
            Pc = linear_trans(K,Xh)
            Xc_poly = Polyhedron(Xc)
            curr_time = (time.time()-start_time)
            msg = str((i+1, len(Xc), Xc_poly.volume(), curr_time))
            msg = msg[1:-1]
            print(msg)
            f.write(msg+'\n')
    except KeyboardInterrupt:
        break

# Save the runtime
end_time = time.time()
total_time_in_ms = (end_time-start_time)*10**3

# msg = str((iter, len(list(Xc.Hrepresentation())), len(list(Xc.vertices())), len(list(Xc.vertices())[0]), total_time_in_ms, error_msg))
# msg = msg[1:-1]
# f.write(msg)

print("\nRuntime: ", total_time_in_ms, file = f)

# Close the file
f.close()

# Saving the plot
try:
    fig = Xc.plot()
    fig_string = "outputs/alg1/ex" + exp_num + ".png"
    fig.save(fig_string)
except:
    pass

# Plotting in notebook
Xc


 ep = 0.200000000000000

Iteration, Num_Ver(Xc), Volume(Xc), Runtime

1, 4, 18, 0.004370927810668945
2, 16, 226.00000000000014, 0.032312870025634766
3, 64, 281.68, 0.06577777862548828
4, 256, 510.16, 0.12518787384033203
5, 1024, 586.3199999999999, 0.3103370666503906
6, 4096, 835.28, 1.005990982055664
7, 16384, 931.9200000000002, 4.015058994293213
8, 65536, 1201.3600000000001, 23.408864974975586


array([[ 1.8, 16. ],
       [ 1.8, 18. ],
       [ 1.8, 16.8],
       ...,
       [-9.8, 13.2],
       [-9.8, 12. ],
       [-9.8, 10. ]])